# Visualing portfolio performance

In [17]:
import sys

# Add the src directory to the path
# If it bugs out, just alternate between ../../ and ../../src
sys.path.append("../../src")

from src.config import app_config
from sqlalchemy import create_engine
from sqlalchemy.orm import Session

ENGINE  = create_engine(app_config.SQLALCHEMY_DATABASE_URI, echo=False, pool_size=20, max_overflow=0)

In [18]:
from src.service import DataService
from src.model import PortfolioHistoryModel, BalanceHistoryModel, CurrencyPairConfigModel
import matplotlib.pyplot as plt

def bot_info(bot: CurrencyPairConfigModel, **params):
    strat_config = ','.join([sc.value for sc in bot.strategy_config])
    output = f"Bot: {bot.id} - {bot.currency_pair} - {bot.strategy.value} ({strat_config}) - {bot.interval.value}"
    for key, val in params.items():
        output += f" - {key}: {val}"
    print(output)

### Bokeh
For nice graphing...

In [20]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool
from bokeh.palettes import Category10, Dark2_5
from bokeh.io import output_notebook

import itertools

output_notebook()

colors = itertools.cycle(Dark2_5)

class BokehGraph:

    WIDTH = 1400
    HEIGHT = 600

    def __init__(self):
        self.figure = figure(width=self.WIDTH , height=self.HEIGHT, x_axis_type="datetime")
        self.figure.add_tools(HoverTool(
            tooltips=[
                ('date', '@x{%F}'),
                ('value', '@y{0.0000}'),
            ],
            formatters={
                '@x': 'datetime',
            },
            mode='vline'
        ))

    def add_line(self, x, y, legend_label, color):
        source = ColumnDataSource(data=dict(x=x, y=y))
        self.figure.line(x="x", y="y", source=source, line_width=2, legend_label=legend_label, color=next(colors))

    def show(self):
        # Hide lines on click
        self.figure.legend.click_policy = "hide"
        show(self.figure)

Loading BokehJS ...

### Bot performance

In [6]:
# Number of active bots
with Session(ENGINE) as session:
    bots = DataService().get_active_bots(session)
    print(len(bots))

219


##### Show profitable bots

In [21]:
from src.service import DataService
from src.model import CurrencyPairConfigModel, OrderModel
from src.enums import OrderSide

with Session(ENGINE) as session:
    users = DataService().get_users(session)
    for user in users:
        bots = user.currency_pair_configs
        profit = 0
        for bot in bots:
            orders = bot.orders
            if len(orders) == 0:
                continue

            latest_order: OrderModel = orders[-1]            
            if latest_order.side.value == OrderSide.BUY.value:
                profit += latest_order.cost - bot.allocated_balance
            else:
                profit += latest_order.amount - bot.allocated_balance
        
        print(f"User Id: {user.id} | Strategy: {bots[0].strategy.value} | Interval: {bots[0].interval.value} - P/L: {profit}") if profit > 0 else None

User Id: 1 | Strategy: Bollinger Bands | Interval: 1m - P/L: 3.7841064699999833
User Id: 23 | Strategy: Relative Strength Index | Interval: 1m - P/L: 1107.686786799999
User Id: 24 | Strategy: Chaikin Oscillator | Interval: 1m - P/L: 514.0774975699997
User Id: 25 | Strategy: Chaikin Oscillator | Interval: 3m - P/L: 7.927767969998968
User Id: 26 | Strategy: Chaikin Oscillator | Interval: 5m - P/L: 460.1089485000002
User Id: 28 | Strategy: Chaikin Oscillator | Interval: 30m - P/L: 1084.45605883
User Id: 32 | Strategy: Commodity Channel Index | Interval: 1m - P/L: 598.8055642300005
User Id: 33 | Strategy: Commodity Channel Index | Interval: 5m - P/L: 732.9403761900003
User Id: 35 | Strategy: Commodity Channel Index | Interval: 30m - P/L: 721.7181488200013
User Id: 36 | Strategy: Commodity Channel Index | Interval: 1h - P/L: 883.5794980299997
User Id: 37 | Strategy: Commodity Channel Index | Interval: 2h - P/L: 96.40561771000012
User Id: 44 | Strategy: Chaikin Oscillator | Interval: 15m - P

**For a particular user**

In [22]:
from src.model import CurrencyPairConfigModel, OrderModel
from src.enums import OrderSide

# The user to check
# be mindful of how many active bots are running.
# E.g. plotting 400 bots is pretty slow
user_id = 46

with Session(ENGINE) as session:
    user = DataService().get_user(session, user_id)
    bots: list[CurrencyPairConfigModel] = user.currency_pair_configs
    
    print("=====================================")
    print(f"User: {user.email}")
    
    # Plot the history of orders for each bot in bokeh
    graph = BokehGraph()
    for i, bot in enumerate(bots):
        orders: list[OrderModel] = bot.orders
        x = [o.created_at for o in orders if o.side.value == OrderSide.BUY.value]
        y = [o.cost for o in orders if o.side.value == OrderSide.BUY.value]
        
        divisor = len(y) if len(y) > 0 else 1
        avg = sum(y) / divisor

        # Skip plotting the losing bots
        if avg < bot.allocated_balance:
            # Disable the bots if desired
            # bot.is_active = False
            # session.commit()
            continue
        
        bot_info(bot, avg=avg)
        graph.add_line(x, y, f"Bot {i} - {bot.currency_pair}", next(colors))

    graph.show()


User: bot46@admin.com
Bot: 906 - ARPAUSDT - Relative Strength Index (1) - 1m - avg: 101419.43570800206
Bot: 913 - AVAUSDT - Relative Strength Index (1) - 1m - avg: 101823.27419778593
Bot: 935 - CELOUSDT - Relative Strength Index (1) - 1m - avg: 106295.4332114966
Bot: 892 - CHRUSDT - Relative Strength Index (1) - 1m - avg: 100835.0789007231
Bot: 908 - CKBUSDT - Relative Strength Index (1) - 1m - avg: 111324.50287121716
Bot: 925 - FISUSDT - Relative Strength Index (1) - 1m - avg: 101691.5540646145
Bot: 920 - FORUSDT - Relative Strength Index (1) - 1m - avg: 102693.70368544871
Bot: 926 - GALUSDT - Relative Strength Index (1) - 1m - avg: 102482.3123216681
Bot: 902 - ILVUSDT - Relative Strength Index (1) - 1m - avg: 100262.90937828754
Bot: 894 - IMXUSDT - Relative Strength Index (1) - 1m - avg: 102413.70461396634
Bot: 931 - KDAUSDT - Relative Strength Index (1) - 1m - avg: 101262.63021612137
Bot: 918 - KNCUSDT - Relative Strength Index (1) - 1m - avg: 103659.58452347295
Bot: 933 - MINAUSDT 